# Librerias

In [26]:
#%pip install langchain_community langchain_text_splitters pypdf langchain_chroma

In [27]:
#%pip install -U langchain-ollama langchain-huggingface

# Carga de PDF's

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./data/Directrices APEC.pdf" #Ruta al archivo PDF
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

In [ ]:
docs

# Embeddings

In [ ]:
%pip install -U sentence-transformers

In [31]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [32]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [33]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

retriever = vectorstore.as_retriever()

In [ ]:
retriever.get_relevant_documents('consentimiento del individuo')

# LLM

In [35]:
import chromadb
from langchain_community.chat_models import ChatOllama
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

persistent_client = chromadb.PersistentClient(path='./database')

In [36]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) #usamos menos tokens que en el anterior debido a que este modelo es inferior
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory='./database')

retriever = vectorstore.as_retriever()

In [ ]:
retriever.get_relevant_documents('consentimiento del individuo')

In [ ]:
llm = ChatOllama(model='llama3.2', temperature=0)

### Crear prompt

In [39]:
system_prompt = (
    "Eres un asistente legal experto en responder preguntas de legalidad. "
    "Usa los siguientes fragmentos de contexto recuperado para proporcionar respuestas precisas, "
    "fundamentadas en principios legales, normativas aplicables o jurisprudencia relevante. "
    "Si no sabes la respuesta, indica que no puedes ofrecer una respuesta definitiva. "
    "Sé claro, directo y profesional, utilizando un máximo de tres oraciones."
    "\n\n"
    "{context}"
)

In [40]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


chain = create_stuff_documents_chain(llm, prompt)
rag = create_retrieval_chain(retriever, chain)

In [41]:
results = rag.invoke({"input": "consentimiento del individuo"})

In [ ]:
results

In [ ]:
results['answer']

In [44]:
chroma_local = Chroma(persist_directory="./database", embedding_function=embeddings)

In [45]:
retriever = chroma_local.as_retriever()

In [ ]:
retriever.get_relevant_documents('información personal')

In [47]:
def prompt():

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ])
    return prompt

In [48]:
def respuesta(pregunta, llm, chroma_db, prompt):
    retriever = chroma_db.as_retriever()

    chain = create_stuff_documents_chain(llm, prompt)
    rag = create_retrieval_chain(retriever, chain)
    
    results = rag.invoke({"input": pregunta})
    return results

In [ ]:
respuesta('Marco de Privacidad de APEC', llm, chroma_local, prompt())